In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
from lib.modules import (
    read_and_window_nursing_session,
    read_nursing_session,
    read_nursing_labels,
    optimization_loop,
    evaluate_loop,
    predict_and_plot_pretty_session
)
from lib.utils import (
    plot_and_save_cm,
    plot_and_save_losses,
    summary,
    get_bouts_smoothed
)
from lib.models import ConvAutoencoderImproved, EncoderClassifierImproved, ConvEncoderClassifierImproved
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
nursing_raw_dir = Path("/home/musa/datasets/nursingv1")
nursing_label_dir = Path("/home/musa/datasets/eating_labels")
WINSIZE = 101
DEVICE = 'cuda:1'
autoencoder_dir = Path('dev/autoencoder4_conv-impr')
encoderclass_dir = Path('dev/encoderclass4_conv')

# Data Processing

In [ ]:
all_sessions = set(range(71))
not_labeled = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 34, 70}
reserved = {15, 43, 45, 51, 55}
sessions = list(all_sessions - not_labeled - reserved)
np.random.seed(10)
train_sessions, test_sessions = train_test_split(sessions, test_size=0.25)

In [ ]:
# train_sessions = [25, 67, 42, 50, 22, 61, 33, 21, 16, 18]
# test_sessions = [58, 62]

Xs = []
ys = []

for session_idx in train_sessions:
    X,y = read_and_window_nursing_session(session_idx, WINSIZE, nursing_raw_dir, nursing_label_dir)

    Xs.append(X)
    ys.append(y)

Xtr = torch.cat(Xs)
ytr = torch.cat(ys)

Xs = []
ys = []

for session_idx in test_sessions:
    X,y = read_and_window_nursing_session(session_idx, WINSIZE, nursing_raw_dir, nursing_label_dir)

    Xs.append(X)
    ys.append(y)

Xte = torch.cat(Xs)
yte = torch.cat(ys)

In [ ]:
nursing_trainloader = DataLoader(TensorDataset(Xtr, ytr), batch_size=64, shuffle=True)
nursing_testloader = DataLoader(TensorDataset(Xte,yte), batch_size=64)

torch.save(nursing_trainloader, 'pytorch_datasets/nursing_trainloader_11-16-23.pt')
torch.save(nursing_testloader, 'pytorch_datasets/nursing_testloader_11-16-23.pt')

# Train

In [4]:
nursing_trainloader = torch.load('pytorch_datasets/nursing_trainloader_11-16-23.pt')
nursing_testloader = torch.load('pytorch_datasets/nursing_testloader_11-16-23.pt')

In [5]:
nursing_trainloader = torch.load('pytorch_datasets/nursing_trainloader-test.pt')
nursing_testloader = torch.load('pytorch_datasets/nursing_testloader-test.pt')

In [6]:
model = ConvEncoderClassifierImproved(WINSIZE, weights_file=autoencoder_dir / 'best_model.pt', freeze=True).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.BCEWithLogitsLoss()

In [10]:
optimization_loop(model, nursing_trainloader, nursing_testloader, criterion, optimizer, 40, DEVICE, patience=20, min_delta=0.001, outdir=Path('dev/test'))

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 27: Train Loss: 0.36761: Dev Loss: 2.4624:  68%|██████▊   | 27/40 [01:49<00:52,  4.05s/it]

Early stopping at epoch 27


# Evaluate


In [ ]:
nursing_trainloader = torch.load('pytorch_datasets/nursing_trainloader_11-16-23.pt')
nursing_testloader = torch.load('pytorch_datasets/nursing_testloader_11-16-23.pt')

In [ ]:
model = EncoderClassifierImproved(WINSIZE, weights_file=autoencoder_dir / 'best_model.pt', freeze=True).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model.load_state_dict(torch.load(encoderclass_dir / 'best_model.pt'))

In [ ]:
ys,metrics = evaluate_loop(model, criterion, nursing_trainloader, DEVICE)
plot_and_save_cm(ys['true'], ys['pred'])
summary(metrics)

In [ ]:
ys,metrics = evaluate_loop(model, criterion, nursing_testloader, DEVICE)
plot_and_save_cm(ys['true'], ys['pred'])
summary(metrics)

In [ ]:
test_session = test_sessions[0]

predict_and_plot_pretty_session(
    session_idx=test_session,
    dim_factor=5,
    datapath=nursing_raw_dir,
    labelpath=nursing_label_dir,
    winsize=WINSIZE,
    model=model,
    criterion=criterion,
    batch_size=64,
    device=DEVICE,
    smooth=False
)